In [82]:
import pandas as pd
import requests, os, pickle
from multiprocessing import Pool
from datetime import date, datetime, timedelta

# 네이버 증권 데이터 함수

### 삼성전자 증권 데이터 불러오기

In [99]:
payload = {"symbol": "005930",
          "requestType": "1",
          "startTime": "20170202",
          "endTime": "20180508",
          "timeframe": "day",}

ajax_url = "https://api.finance.naver.com/siseJson.naver?"
head = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.44"}

r = requests.post(ajax_url, data = payload, headers = head)

data = eval(" ".join(r.text.strip().split()))
df = pd.DataFrame(data[1:], columns=data[0])
df.to_csv("./data/삼성전자.csv", encoding = 'utf-8-sig', index=False)

In [100]:
df

,날짜,시가,고가,저가,종가,거래량,외국인소진율
0,20170202,39600,39660,39200,39360,300586,50.72
1,20170203,39400,39500,39180,39460,207727,50.70
2,20170206,39580,39660,39140,39560,177214,50.69
3,20170207,39560,39580,38760,38820,280836,50.65
4,20170208,38740,38779,38200,38400,341859,50.61
...,...,...,...,...,...,...,...
303,20180430,0,0,0,53000,0,52.06
304,20180502,0,0,0,53000,0,52.06
305,20180503,0,0,0,53000,0,0.00
306,20180504,53000,53900,51800,51900,39565391,52.79


### 날짜 변환 하여 데이터 불러오기

In [101]:
# 현재날짜 타입 datetime으로 변환
datetime.strftime(date.today(), '%Y%m%d')
str(date.today()).replace("-", "")

'20220421'

In [102]:
# 한달 전의 날짜
str(date.today() - timedelta(days = 30)).replace("-", "")

'20220322'

In [103]:
payload['startTime'] = str(date.today()).replace("-", "")
payload['endTime'] = str(date.today() - timedelta(days = 30)).replace("-", "")

In [104]:
ajax_url = "https://api.finance.naver.com/siseJson.naver?symbol=005930&requestType=1&startTime=20210727&endTime=20220421&timeframe=day"
head = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.44"}

r = requests.post(ajax_url, data = payload, headers = head)

data = eval(" ".join(r.text.strip().split()))
df = pd.DataFrame(data[1:], columns=data[0])
# df.to_csv("./data/삼성전자.csv", encoding = 'utf-8-sig', index=False)

In [105]:
df

,날짜,시가,고가,저가,종가,거래량,외국인소진율
0,20210727,79200,79400,78500,78500,11427693,53.27
1,20210728,78300,79200,78100,79200,12743417,53.22
2,20210729,78800,79400,78800,79000,14360453,53.19
3,20210730,78900,78900,78500,78500,13823411,53.17
4,20210802,79200,79500,78700,79300,11739124,53.15
...,...,...,...,...,...,...,...
176,20220415,67200,67300,66500,66600,13176415,51.21
177,20220418,66500,67100,66100,66700,10119203,51.19
178,20220419,67100,68000,67000,67300,12959434,51.17
179,20220420,67000,67400,66500,67400,16693293,51.14


### 주식 데이터 불러오는 함수 생성

In [107]:
# stock.py 파일로 따로 생성
def get_stock(code, startTime=None, endTime=None, type_='day'):
    payload = {  "requestType": "1",
          "timeframe": "day",}
    
    ajax_url = "https://api.finance.naver.com/siseJson.naver?"
    head = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.44"}
    
    payload['symbol']= code 
    payload['endTime'] = datetime.strftime(date.today(), "%Y%m%d")
    payload['startTime'] = str(date.today() - timedelta(days=30)).replace("-", "")
    
    r = requests.post(ajax_url, data=payload, headers= head)
    
    data = eval(" ".join(r.text.strip().split()))
    
    df = pd.DataFrame(data[1:], columns=data[0])
    
    if os.path.isdir("./data") == False:
        os.mkdir("./data")
    df.to_csv(f"./data/{code}.csv", encoding='utf-8-sig', index=False)

if __name__ == "__main__":
    get_stock('005930')